<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/010_Mask2Former_ObtenerDatos_Avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mahotas scikit-image opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.7 MB/s eta 0:00:00


In [2]:
### Librería principal para computación con tensores y modelos de deep learning (PyTorch)
import torch
### Manejo de datos en formato JSON
import json
### Funciones relacionadas con el tiempo (ej. timestamps, delays)
import time
### Interacción con el sistema operativo (rutas, archivos, etc.)
import os
### Manejo de fechas y horas con precisión
from datetime import datetime
### Manejo de rutas de archivos de forma más robusta que con strings
from pathlib import Path
### Generación de hashes (útil para verificar integridad o crear IDs únicos)
import hashlib
### Decorador para crear clases de datos de forma concisa
from dataclasses import dataclass

import numpy as np
import cv2
from PIL import Image, ImageStat
import mahotas as mh
from skimage import feature, filters, segmentation, measure
from skimage.color import rgb2gray, rgb2hsv, rgb2lab
from sklearn.cluster import KMeans
from typing import Dict, List, Any, Optional

### Importación de modelos y procesadores de imágenes desde Hugging Face Transformers ###
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation
### AutoImageProcessor: preprocesamiento automático de imágenes para modelos específicos
### AutoModelForUniversalSegmentation: modelo para segmentación semántica de imágenes

### Visualización de datos e imágenes
import matplotlib.pyplot as plt
### Barra de progreso para loops largos (mejora experiencia visual en ejecución)
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

### Montaje de Google Drive en entorno Colab para acceder a archivos almacenados en la nube ###
from google.colab import drive
### Solicita autorización para montar Google Drive en la ruta especificada
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
### Clase de configuración centralizada para definir rutas, parámetros y modelos utilizados en el proyecto ###
@dataclass
class Config:

    ### Ruta al conjunto de datos de entrada (imágenes a procesar) ###
    DATASET_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former/imagenes")

    ### Ruta donde se guardarán los resultados del procesamiento (segmentaciones, métricas, etc.) ###
    OUTPUT_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former/resultados")

    ### Diccionario de umbrales predefinidos para filtrar o evaluar resultados de segmentación.
    ### Cada clave representa un perfil de sensibilidad con descripción detallada:
    UMBRALES = {
        'ultra_sensible': {
            'valores': [0.0001, 0.001, 0.01, 0.1],
            'descripcion': 'Detecta cambios mínimos - Máxima sensibilidad'
        },
        'alta_sensibilidad': {
            'valores': [0.001, 0.01, 0.05, 0.1, 0.3],
            'descripcion': 'Sensibilidad alta para detección temprana'
        },
        'sensibilidad_media': {
            'valores': [0.01, 0.1, 0.3, 0.5],
            'descripcion': 'Balance entre precisión y recall'
        },
        'baja_sensibilidad': {
            'valores': [0.3, 0.5, 0.7],
            'descripcion': 'Solo detecciones muy confiables'
        }
    }

    ### Lista de modelos de segmentación con información detallada ###
    MODELOS_INFO = [
        {
            'nombre_hf': "facebook/mask2former-swin-large-coco-instance",
            'nombre_corto': "swin-large-coco-instance",
            'tipo': 'instancia',
            'dataset': 'COCO',
            'arquitectura': 'Swin-Large',
            'descripcion': 'Segmentación por instancia - Mayor precisión'
        },
        {
            'nombre_hf': "facebook/mask2former-swin-base-ade-semantic",
            'nombre_corto': "swin-base-ade-semantic",
            'tipo': 'semantico',
            'dataset': 'ADE20K',
            'arquitectura': 'Swin-Base',
            'descripcion': 'Segmentación semántica - Balance eficiencia/precisión'
        },
        {
            'nombre_hf': "facebook/mask2former-swin-small-coco-instance",
            'nombre_corto': "swin-small-coco-instance",
            'tipo': 'instancia',
            'dataset': 'COCO',
            'arquitectura': 'Swin-Small',
            'descripcion': 'Segmentación por instancia - Más rápido'
        }
    ]

### Se crea una instancia de la clase Config para acceder a rutas, modelos y parámetros definidos ###
config = Config()

### Se asegura que la carpeta de salida exista; si no existe, se crea automáticamente.
### El parámetro `exist_ok=True` evita errores si la carpeta ya está creada ###
os.makedirs(config.OUTPUT_PATH, exist_ok=True)

In [10]:
### Clase que agrupa funciones utilitarias reutilizables para el procesamiento de imágenes y manejo de archivos ###
class Utils:

    ### Carga todas las imágenes desde una ruta dada, buscando recursivamente archivos con extensiones válidas.
    ### Parámetros:
    ### - ruta (str): ruta base donde buscar imágenes
    ### Retorna:
    ### - Lista de rutas (str) de imágenes encontradas
    @staticmethod
    def cargar_imagenes(ruta: str) -> List[str]:
        path = Path(ruta)
        extensiones = (".jpg", ".png", ".jpeg")
        imagenes = [str(p) for p in path.glob("**/*") if p.suffix.lower() in extensiones]
        print(f"📁 Encontradas {len(imagenes)} imágenes en {ruta}")
        return imagenes

    ### Abre una imagen desde disco, la convierte a RGB y la redimensiona si excede el tamaño máximo.
    ### Parámetros:
    ### - ruta (str): ruta del archivo de imagen
    ### - max_size (int): tamaño máximo permitido para ancho/alto (por defecto 1024)
    ### Retorna:
    ### - Imagen PIL preparada para procesamiento
    @staticmethod
    def preparar_imagen(ruta: str, max_size: int = 1024) -> Image.Image:
        img = Image.open(ruta).convert("RGB")
        if max(img.size) > max_size:
            img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
        return img

    ### Calcula un hash MD5 de los contenidos binarios del archivo.
    ### Útil para identificar imágenes de forma única o evitar duplicados.
    ### Parámetros:
    ### - ruta (str): ruta del archivo
    ### Retorna:
    ### - Cadena con los primeros 8 caracteres del hash MD5
    @staticmethod
    def calcular_hash(ruta: str) -> str:
        return hashlib.md5(open(ruta, 'rb').read()).hexdigest()[:8]

    ### Guarda un objeto Python (lista, diccionario, etc.) en un archivo JSON.
    ### Parámetros:
    ### - datos (Any): objeto a guardar
    ### - archivo (str): ruta del archivo destino
    ### No retorna nada, pero imprime confirmación en consola.
    @staticmethod
    def guardar_json(datos: Any, archivo: str) -> None:
        with open(archivo, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)
        print(f"💾 Archivo guardado: {os.path.basename(archivo)}")

    ### Nueva función para crear nombres de archivo descriptivos
    @staticmethod
    def crear_nombre_archivo(modelo_info: Dict, umbral_config: str, timestamp: str) -> str:
        """
        Crea un nombre de archivo descriptivo con toda la información relevante
        """
        tipo_modelo = modelo_info['tipo']
        dataset = modelo_info['dataset'].lower()
        arquitectura = modelo_info['arquitectura'].lower().replace('-', '')

        # Formato: mask2former_{tipo}_{dataset}_{arquitectura}_{umbral_config}_{timestamp}.json
        nombre = f"mask2former_{tipo_modelo}_{dataset}_{arquitectura}_{umbral_config}_{timestamp}.json"
        return nombre

In [4]:
### Clase DetectorPersonas ###
class DetectorPersonas:
    ### Constructor de la clase con logging mejorado
    def __init__(self, modelo_info: Dict):
        ### Guarda información completa del modelo
        self.modelo_info = modelo_info
        self.modelo_name = modelo_info['nombre_hf']

        ### Selección automática del dispositivo: GPU si está disponible, de lo contrario CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Logging mejorado con información clara del modelo
        print(f"\n🤖 CARGANDO MODELO:")
        print(f"   📌 Nombre: {modelo_info['nombre_corto']}")
        print(f"   🔧 Tipo: {modelo_info['tipo'].upper()}")
        print(f"   🏗️  Arquitectura: {modelo_info['arquitectura']}")
        print(f"   📊 Dataset: {modelo_info['dataset']}")
        print(f"   💡 Descripción: {modelo_info['descripcion']}")
        print(f"   🖥️  Dispositivo: {self.device}")

        ### Carga el procesador de imágenes asociado al modelo desde Hugging Face
        print("   ⏳ Cargando procesador...")
        self.processor = AutoImageProcessor.from_pretrained(self.modelo_name)

        ### Carga el modelo de segmentación universal desde Hugging Face
        print("   ⏳ Cargando modelo...")
        self.model = AutoModelForUniversalSegmentation.from_pretrained(self.modelo_name)

        ### Mueve el modelo al dispositivo seleccionado y lo pone en modo evaluación
        self.model.to(self.device)
        self.model.eval()

        ### Determina si el modelo es de segmentación semántica
        self.es_semantico = (modelo_info['tipo'] == 'semantico')

        ### Si el modelo tiene un diccionario de clases (id2label), se intenta identificar la clase "persona"
        if hasattr(self.model.config, 'id2label'):
            self.id2label = self.model.config.id2label
            print(f"   🏷️  Clases disponibles: {len(self.id2label)}")
            print(f"   🏷️  Clase por defecto (ID 0): {self.id2label.get(0, 'DESCONOCIDA')}")

            ### Búsqueda de la clase "persona" en el diccionario de etiquetas
            self.clase_persona = 0
            for clase_id, nombre in self.id2label.items():
                if 'person' in nombre.lower() or 'people' in nombre.lower():
                    self.clase_persona = clase_id
                    print(f"   👤 Clase persona encontrada: ID {clase_id} = '{nombre}'")
                    break
            else:
                print(f"   ⚠️  Usando clase por defecto (ID 0) como 'persona'")
        else:
            ### Si no hay diccionario de clases, se asume por defecto que la clase "persona" es la 0
            self.clase_persona = 0
            print(f"   ⚠️  Sin diccionario de clases, usando ID 0 como 'persona'")

        print(f"   ✅ Modelo cargado exitosamente\n")

    ### Método detectar_en_imagen con mejor logging ###
    def detectar_en_imagen(self, imagen: Image.Image, umbrales: List[float]) -> Dict:
        ### Obtiene dimensiones de la imagen (orden correcto: width, height)
        w, h = imagen.size

        ### Preprocesa la imagen y la convierte en tensores para el modelo
        inputs = self.processor(images=imagen, return_tensors="pt").to(self.device)

        ### Ejecuta la inferencia sin cálculo de gradientes y mide el tiempo de ejecución
        inicio = time.time()
        with torch.no_grad():
            outputs = self.model(**inputs)
        tiempo_ms = (time.time() - inicio) * 1000

        ### Diccionario para almacenar resultados por umbral
        resultados = {'tiempo_inferencia_ms': tiempo_ms}

        ### Post-procesamiento para modelos semánticos
        if self.es_semantico:
            try:
                ### Obtiene la máscara semántica con clases por píxel
                resultado_semantico = self.processor.post_process_semantic_segmentation(
                    outputs, target_sizes=[(h, w)]
                )[0]

                ### Extrae clases únicas presentes en la imagen
                unique_classes = torch.unique(resultado_semantico)

                ### Para cada umbral, se calcula si hay presencia significativa de la clase "persona"
                for umbral in umbrales:
                    ### Crea una máscara booleana donde los píxeles coinciden con la clase "persona"
                    persona_mask = (resultado_semantico == self.clase_persona)

                    ### Cuenta el número de píxeles de clase "persona"
                    persona_pixels = persona_mask.sum().item()
                    total_pixels = resultado_semantico.numel()

                    ### Calcula el porcentaje actual de píxeles de clase "persona"
                    porcentaje_actual = (persona_pixels / total_pixels) * 100
                    porcentaje_minimo = umbral * 100

                    ### Se considera que hay personas si se supera el umbral y hay más de 50 píxeles
                    personas = 1 if porcentaje_actual >= porcentaje_minimo and persona_pixels > 50 else 0

                    resultados[f'umbral_{umbral}'] = {
                        'personas': personas,
                        'total': len(unique_classes),
                        'todas_clases': unique_classes.tolist(),
                        'persona_pixels': persona_pixels,
                        'porcentaje_persona': porcentaje_actual,
                        'scores_personas': [1.0] if personas > 0 else [],
                        'todos_scores': [1.0] * len(unique_classes),
                        'max_score': 1.0 if len(unique_classes) > 0 else 0.0
                    }

            ### En caso de error, se registra en el resultado por cada umbral
            except Exception as e:
                for umbral in umbrales:
                    resultados[f'umbral_{umbral}'] = {
                        'error': str(e), 'personas': 0, 'total': 0,
                        'scores_personas': [], 'todos_scores': [], 'max_score': 0.0
                    }

        ### Post-procesamiento para modelos de instancia
        else:
            for umbral in umbrales:
                try:
                    ### Aplica post-procesamiento con el umbral de score especificado
                    resultado = self.processor.post_process_instance_segmentation(
                        outputs, target_sizes=[(h, w)], threshold=umbral
                    )[0]

                    ### Extrae etiquetas y scores de las detecciones
                    labels = resultado.get("labels", torch.tensor([]))
                    scores = resultado.get("scores", torch.tensor([]))

                    ### Filtra detecciones que corresponden a la clase "persona"
                    personas = sum(1 for l in labels if int(l.item()) == self.clase_persona)

                    ### Extrae scores de las detecciones de personas
                    scores_personas = [
                        float(s.item()) for l, s in zip(labels, scores)
                        if int(l.item()) == self.clase_persona
                    ]

                    ### Extrae todas las clases y scores detectados
                    todas_clases = [int(l.item()) for l in labels]
                    todos_scores = [float(s.item()) for s in scores]

                    resultados[f'umbral_{umbral}'] = {
                        'personas': personas,
                        'total': len(labels),
                        'scores_personas': scores_personas,
                        'todas_clases': todas_clases,
                        'todos_scores': todos_scores,
                        'max_score': max(todos_scores) if todos_scores else 0.0
                    }

                ### En caso de error, se registra en el resultado por cada umbral
                except Exception as e:
                    resultados[f'umbral_{umbral}'] = {
                        'error': str(e), 'personas': 0, 'total': 0,
                        'scores_personas': [], 'todos_scores': [], 'max_score': 0.0
                    }

        return resultados

    ### Método para liberar memoria del modelo y procesador ###
    def limpiar(self):
        print(f"🧹 Liberando memoria del modelo {self.modelo_info['nombre_corto']}")
        del self.model, self.processor
        torch.cuda.empty_cache()

In [5]:
### Clase ProcesadorResultados ###
class ProcesadorResultados:
    def __init__(self, output_path: str):
        self.output_path = Path(output_path)

    ### Método para procesar una imagen completa con mejor logging ###
    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonas,
                        umbrales: List[float]) -> Optional[Dict]:
        inicio = time.time()

        try:
            ### Prepara la imagen para el modelo
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_img = Utils.calcular_hash(ruta_imagen)
            img_array = np.array(imagen)
            gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

            ### Extrae información básica de la imagen
            info_imagen = {
                'archivo': os.path.basename(ruta_imagen),
                'hash': hash_img,
                'tamaño': imagen.size,
                'brillo_promedio': float(np.mean(gray))
            }

            ### Ejecuta la detección de personas
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            ### Calcula el tiempo total de procesamiento
            tiempo_total = (time.time() - inicio) * 1000

            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': info_imagen,
                'deteccion': resultados_deteccion,
                'tiempo_total_ms': tiempo_total,
                'modelo': detector.modelo_info,  # Información completa del modelo
                'exitoso': True
            }

        except Exception as e:
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {'archivo': os.path.basename(ruta_imagen)},
                'error': str(e),
                'exitoso': False
            }

    ### Método mejorado para mostrar resumen estadístico ###
    def mostrar_resumen(self, resultados: List[Dict], umbral_config: str) -> None:
        exitosos = sum(1 for r in resultados if r.get('exitoso', False))

        print(f"\n📊 RESUMEN DE PROCESAMIENTO:")
        print(f"   ✅ Imágenes procesadas exitosamente: {exitosos}/{len(resultados)}")

        if exitosos == 0:
            print("   ❌ No hay resultados exitosos para analizar")
            return

        # Obtener información del modelo del primer resultado exitoso
        modelo_info = None
        for r in resultados:
            if r.get('exitoso', False):
                modelo_info = r.get('modelo', {})
                break

        if modelo_info:
            print(f"   🤖 Modelo usado: {modelo_info.get('nombre_corto', 'N/A')}")
            print(f"   🔧 Tipo: {modelo_info.get('tipo', 'N/A').upper()}")

        # Obtener descripción de umbrales
        umbral_info = config.UMBRALES.get(umbral_config, {})
        print(f"   ⚙️  Configuración umbrales: {umbral_info.get('descripcion', umbral_config)}")

        ### Itera sobre los umbrales para mostrar estadísticas
        umbrales = umbral_info.get('valores', [])
        print(f"\n📈 RESULTADOS POR UMBRAL:")

        for umbral in umbrales:
            personas_detectadas = []

            ### Recorre cada resultado exitoso
            for r in resultados:
                if r.get('exitoso', False):
                    deteccion = r.get('deteccion', {})
                    umbral_data = deteccion.get(f'umbral_{umbral}', {})
                    personas_detectadas.append(umbral_data.get('personas', 0))

            ### Calcula estadísticas
            if personas_detectadas:
                total_personas = sum(personas_detectadas)
                imagenes_con_personas = sum(1 for p in personas_detectadas if p > 0)
                porcentaje = (imagenes_con_personas / len(personas_detectadas)) * 100

                print(f"   🎯 Umbral {umbral:7.4f}: {total_personas:3d} personas | {imagenes_con_personas:3d}/{len(personas_detectadas)} imágenes ({porcentaje:5.1f}%)")


In [6]:
class ExtractorCaracteristicasAvanzado:
    """
    Extractor de características de imágenes usando librerías especiales
    para análisis completo.
    """

    def __init__(self):
        print("🔧 Inicializando Extractor de Características Avanzado")
        print("   📚 Librerías: Mahotas, Scikit-image, OpenCV")

    def analizar_imagen_completa(self, imagen: Image.Image, ruta: str) -> Dict[str, Any]:
        """
        Análisis completo usando librerías especializadas

        Args:
            imagen: Imagen PIL en formato RGB
            ruta: Ruta del archivo original

        Returns:
            Diccionario completo con características extraídas
        """
        # Conversiones básicas
        img_array = np.array(imagen)
        img_gray = rgb2gray(img_array)
        img_hsv = rgb2hsv(img_array)
        img_lab = rgb2lab(img_array)

        # Convertir a uint8 para mahotas
        img_gray_uint8 = (img_gray * 255).astype(np.uint8)

        caracteristicas = {
            'metadatos_basicos': self._extraer_metadatos_basicos(imagen, ruta),
            'color_y_paleta': self._analizar_color_avanzado(img_array, img_hsv, img_lab),
            'texturas_mahotas': self._extraer_texturas_mahotas(img_gray_uint8),
            'texturas_skimage': self._extraer_texturas_skimage(img_gray),
            'caracteristicas_geometricas': self._analizar_geometria_avanzada(img_gray_uint8),
            'multiscale_features': self._extraer_multiscale_features(img_gray),
            'propiedades_regionales': self._analizar_propiedades_regionales(img_gray),
            'descriptores_locales': self._extraer_descriptores_locales(img_gray_uint8)
        }

        return caracteristicas

    def _extraer_metadatos_basicos(self, imagen: Image.Image, ruta: str) -> Dict:
        """Metadatos básicos de la imagen"""
        w, h = imagen.size
        return {
            'dimensiones': {'ancho': w, 'alto': h},
            'aspecto_ratio': round(w / h, 3),
            'megapixeles': round((w * h) / 1000000, 2),
            'orientacion': 'horizontal' if w > h else 'vertical' if h > w else 'cuadrada',
            'formato': ruta.split('.')[-1].lower() if '.' in ruta else 'desconocido'
        }

    def _analizar_color_avanzado(self, img_rgb: np.ndarray, img_hsv: np.ndarray, img_lab: np.ndarray) -> Dict:
        """Análisis de color usando sklearn para paleta dominante"""
        # Paleta dominante con K-means
        pixels = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
        kmeans.fit(pixels)

        colores_dominantes = kmeans.cluster_centers_.astype(int).tolist()
        proporciones = np.bincount(kmeans.labels_) / len(kmeans.labels_)

        # Estadísticas básicas por canal
        stats_color = {
            'paleta_dominante': colores_dominantes,
            'proporciones_colores': proporciones.tolist(),
            'estadisticas_rgb': {
                'media': np.mean(img_rgb, axis=(0,1)).tolist(),
                'std': np.std(img_rgb, axis=(0,1)).tolist(),
                'rango': {
                    'min': np.min(img_rgb, axis=(0,1)).tolist(),
                    'max': np.max(img_rgb, axis=(0,1)).tolist()
                }
            },
            'hsv_global': {
                'hue_medio': float(np.mean(img_hsv[:,:,0])),
                'saturacion_media': float(np.mean(img_hsv[:,:,1])),
                'valor_medio': float(np.mean(img_hsv[:,:,2]))
            },
            'lab_luminancia': {
                'L_medio': float(np.mean(img_lab[:,:,0])),
                'a_medio': float(np.mean(img_lab[:,:,1])),
                'b_medio': float(np.mean(img_lab[:,:,2]))
            }
        }

        return stats_color

    def _extraer_texturas_mahotas(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Mahotas"""
        try:
            # Características Haralick (muy utilizadas en análisis de texturas)
            haralick_features = mh.features.haralick(img_gray, return_mean=True)

            # Local Binary Pattern
            lbp = mh.features.lbp(img_gray, radius=1, points=8, ignore_zeros=False)

            # Características Zernike (descriptores de forma)
            try:
                zernike_features = mh.features.zernike_moments(img_gray, radius=21)
            except:
                zernike_features = np.zeros(25)  # Fallback si falla

            # Threshold de Otsu
            otsu_threshold = mh.otsu(img_gray)

            # Características adicionales
            pftas = mh.features.pftas(img_gray)  # Parameter-free threshold adjacency statistics

            return {
                'haralick_features': haralick_features.tolist(),
                'lbp_histogram': np.histogram(lbp, bins=50)[0].tolist(),
                'zernike_moments': zernike_features.tolist(),
                'otsu_threshold': float(otsu_threshold),
                'pftas': pftas.tolist()
            }

        except Exception as e:
            return {
                'error': f"Error en Mahotas: {str(e)}",
                'haralick_features': [],
                'lbp_histogram': [],
                'zernike_moments': [],
                'otsu_threshold': 0.0,
                'pftas': []
            }

    def _extraer_texturas_skimage(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Scikit-image"""
        try:
            # Local Binary Pattern con scikit-image
            lbp_skimage = feature.local_binary_pattern(img_gray, P=8, R=1, method='uniform')

            # GLCM (Gray Level Co-occurrence Matrix) features
            img_scaled = (img_gray * 255).astype(np.uint8)
            glcm = feature.graycomatrix(img_scaled, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                      levels=256, symmetric=True, normed=True)

            # Propiedades GLCM
            contrast = feature.graycoprops(glcm, 'contrast').mean()
            dissimilarity = feature.graycoprops(glcm, 'dissimilarity').mean()
            homogeneity = feature.graycoprops(glcm, 'homogeneity').mean()
            energy = feature.graycoprops(glcm, 'energy').mean()
            correlation = feature.graycoprops(glcm, 'correlation').mean()

            return {
                'lbp_uniform_histogram': np.histogram(lbp_skimage, bins=10)[0].tolist(),
                'glcm_properties': {
                    'contrast': float(contrast),
                    'dissimilarity': float(dissimilarity),
                    'homogeneity': float(homogeneity),
                    'energy': float(energy),
                    'correlation': float(correlation)
                }
            }

        except Exception as e:
            return {
                'error': f"Error en Scikit-image: {str(e)}",
                'lbp_uniform_histogram': [],
                'glcm_properties': {}
            }

    def _analizar_geometria_avanzada(self, img_gray: np.ndarray) -> Dict:
        """Análisis geométrico usando mahotas y scikit-image"""
        try:
            # Detección de bordes con diferentes métodos
            edges_canny = feature.canny(img_gray / 255.0)

            # Usando mahotas para bordes
            edges_sobel = mh.sobel(img_gray)

            # Detección de esquinas
            corners = feature.corner_harris(img_gray / 255.0)
            corner_peaks = feature.corner_peaks(corners, min_distance=5)

            # Análisis de forma usando momentos
            moments = measure.moments(img_gray)
            centroid = measure.centroid(img_gray)

            return {
                'bordes_canny': float(np.sum(edges_canny)),
                'bordes_sobel_intensidad': float(np.mean(edges_sobel)),
                'num_corners': len(corner_peaks),
                'centroide': [float(centroid[0]), float(centroid[1])],
                'momentos_hu': measure.moments_hu(moments).tolist()
            }

        except Exception as e:
            return {
                'error': f"Error en análisis geométrico: {str(e)}",
                'bordes_canny': 0.0,
                'bordes_sobel_intensidad': 0.0,
                'num_corners': 0,
                'centroide': [0.0, 0.0],
                'momentos_hu': []
            }

    def _extraer_multiscale_features(self, img_gray: np.ndarray) -> Dict:
        """Características multi-escala usando scikit-image"""
        try:
            # Características básicas multi-escala
            features_multiscale = feature.multiscale_basic_features(
                img_gray,
                intensity=True,
                edges=True,
                texture=True,
                sigma_min=0.5,
                sigma_max=8
            )

            # Estadísticas de las características multi-escala
            return {
                'num_features': features_multiscale.shape[-1],
                'feature_means': np.mean(features_multiscale, axis=(0,1)).tolist(),
                'feature_stds': np.std(features_multiscale, axis=(0,1)).tolist()
            }

        except Exception as e:
            return {
                'error': f"Error en features multiscale: {str(e)}",
                'num_features': 0,
                'feature_means': [],
                'feature_stds': []
            }

    def _analizar_propiedades_regionales(self, img_gray: np.ndarray) -> Dict:
        """Análisis de propiedades regionales usando segmentación"""
        try:
            # Segmentación usando SLIC
            segments = segmentation.slic(img_gray, n_segments=100, compactness=10)

            # Propiedades de regiones
            regions = measure.regionprops(segments, intensity_image=img_gray)

            if regions:
                areas = [r.area for r in regions]
                eccentricities = [r.eccentricity for r in regions]
                intensities = [r.mean_intensity for r in regions]

                return {
                    'num_regiones': len(regions),
                    'area_promedio': float(np.mean(areas)),
                    'excentricidad_promedio': float(np.mean(eccentricities)),
                    'intensidad_promedio_regiones': float(np.mean(intensities)),
                    'variabilidad_areas': float(np.std(areas)),
                    'variabilidad_intensidades': float(np.std(intensities))
                }
            else:
                return {'num_regiones': 0}

        except Exception as e:
            return {
                'error': f"Error en análisis regional: {str(e)}",
                'num_regiones': 0
            }

    def _extraer_descriptores_locales(self, img_gray: np.ndarray) -> Dict:
        """Descriptores locales usando OpenCV integrado"""
        try:
            # Conversión para OpenCV
            img_cv = img_gray.copy()

            # Detección de puntos clave con diferentes detectores
            # ORB (orientación y escala invariante)
            orb = cv2.ORB_create(nfeatures=100)
            keypoints_orb = orb.detect(img_cv, None)

            # FAST corners
            fast = cv2.FastFeatureDetector_create()
            keypoints_fast = fast.detect(img_cv, None)

            # BRIEF descriptors (si hay keypoints)
            brief = cv2.xfeatures2d.BriefDescriptorExtractor_create() if hasattr(cv2, 'xfeatures2d') else None

            return {
                'orb_keypoints': len(keypoints_orb),
                'fast_keypoints': len(keypoints_fast),
                'keypoint_density': (len(keypoints_orb) + len(keypoints_fast)) / (img_gray.shape[0] * img_gray.shape[1])
            }

        except Exception as e:
            return {
                'error': f"Error en descriptores locales: {str(e)}",
                'orb_keypoints': 0,
                'fast_keypoints': 0,
                'keypoint_density': 0.0
            }


class ProcesadorResultadosConLibrerias(ProcesadorResultados):

    def __init__(self, output_path: str):
        super().__init__(output_path)
        self.extractor = ExtractorCaracteristicasAvanzado()
        print("✅ Procesador con librerías especializadas inicializado")

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonas,
                       umbrales: List[float]) -> Optional[Dict]:

        inicio = time.time()

        try:
            # Preparar imagen
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_img = Utils.calcular_hash(ruta_imagen)

            # Extracción completa de características usando librerías
            print(f"   🔍 Extrayendo características avanzadas...")
            caracteristicas_avanzadas = self.extractor.analizar_imagen_completa(imagen, ruta_imagen)

            # Detección de personas (original)
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            tiempo_total = (time.time() - inicio) * 1000

            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {
                    'archivo': os.path.basename(ruta_imagen),
                    'hash': hash_img,
                    'ruta_completa': ruta_imagen,
                    'caracteristicas_avanzadas': caracteristicas_avanzadas
                },
                'deteccion': resultados_deteccion,
                'tiempo_total_ms': tiempo_total,
                'modelo': detector.modelo_info,
                'version_extractor': 'librerias_especializadas',
                'exitoso': True
            }

        except Exception as e:
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {
                    'archivo': os.path.basename(ruta_imagen),
                    'ruta_completa': ruta_imagen
                },
                'error': str(e),
                'exitoso': False
            }

    def mostrar_resumen_avanzado(self, resultados: List[Dict], umbral_config: str) -> None:
        """
        Resumen que incluye estadísticas de las características extraídas
        """
        super().mostrar_resumen(resultados, umbral_config)

        exitosos = [r for r in resultados if r.get('exitoso', False)]
        if not exitosos:
            return

        print(f"\n🧬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:")

        # Recopilar estadísticas de características
        haralick_means = []
        glcm_contrasts = []
        num_corners = []

        for r in exitosos:
            carac = r['imagen'].get('caracteristicas_avanzadas', {})

            # Características Haralick
            texturas_mh = carac.get('texturas_mahotas', {})
            if 'haralick_features' in texturas_mh and texturas_mh['haralick_features']:
                haralick_means.append(np.mean(texturas_mh['haralick_features']))

            # GLCM contrast
            texturas_sk = carac.get('texturas_skimage', {})
            glcm_props = texturas_sk.get('glcm_properties', {})
            if 'contrast' in glcm_props:
                glcm_contrasts.append(glcm_props['contrast'])

            # Corners
            geom = carac.get('caracteristicas_geometricas', {})
            if 'num_corners' in geom:
                num_corners.append(geom['num_corners'])

        if haralick_means:
            print(f"   📊 Haralick promedio: {np.mean(haralick_means):.3f} ± {np.std(haralick_means):.3f}")
        if glcm_contrasts:
            print(f"   ⚡ GLCM Contrast: {np.mean(glcm_contrasts):.3f} ± {np.std(glcm_contrasts):.3f}")
        if num_corners:
            print(f"   📐 Esquinas promedio: {np.mean(num_corners):.1f} ± {np.std(num_corners):.1f}")

        print(f"   🔬 Usando: Mahotas + Scikit-image + OpenCV")

In [11]:
def ejecutar_con_librerias_especializadas(modelo_idx: int = 0, umbral_config: str = 'alta_sensibilidad'):

    if modelo_idx >= len(config.MODELOS_INFO):
        print(f"❌ Modelo index inválido. Máximo: {len(config.MODELOS_INFO)-1}")
        return

    if umbral_config not in config.UMBRALES:
        print(f"❌ Configuración inválida. Disponibles: {list(config.UMBRALES.keys())}")
        return

    modelo_info = config.MODELOS_INFO[modelo_idx]
    umbral_info = config.UMBRALES[umbral_config]

    print(f"\n{'='*100}")
    print(f"🚀 EVALUACIÓN CON LIBRERÍAS ESPECIALIZADAS")
    print(f"{'='*100}")
    print(f"📋 CONFIGURACIÓN:")
    print(f"   🤖 Modelo: {modelo_info['nombre_corto']}")
    print(f"   🔧 Tipo: {modelo_info['tipo'].upper()}")
    print(f"   📚 Librerías: Mahotas + Scikit-image + OpenCV")
    print(f"   ⚙️  Umbrales: {umbral_info['descripcion']}")

    # Cargar imágenes
    imagenes = Utils.cargar_imagenes(config.DATASET_PATH)
    if not imagenes:
        print("❌ No se encontraron imágenes para procesar")
        return

    umbrales = umbral_info['valores']

    detector = DetectorPersonas(modelo_info)
    procesador = ProcesadorResultadosConLibrerias(config.OUTPUT_PATH)

    print(f"{'='*100}")
    print(f"🔄 PROCESANDO {len(imagenes)} IMÁGENES")
    print(f"{'='*100}")

    resultados = []
    tiempo_inicio_total = time.time()

    for i, ruta in enumerate(tqdm(imagenes, desc="Procesando con librerías especializadas")):
        nombre_archivo = os.path.basename(ruta)
        print(f"\n📷 [{i+1:3d}/{len(imagenes):3d}] {nombre_archivo}")

        resultado = procesador.procesar_imagen(ruta, detector, umbrales)

        if resultado and resultado.get('exitoso', False):
            resultados.append(resultado)

            # Mostrar info básica
            deteccion = resultado['deteccion']
            mejor_umbral = min(umbrales)
            datos = deteccion.get(f'umbral_{mejor_umbral}', {})
            personas = datos.get('personas', 0)
            tiempo_img = deteccion.get('tiempo_inferencia_ms', 0)

            print(f"   ✅ {personas:2d} personas | {tiempo_img:.1f}ms | Características ✓")

    # Guardar resultados
    tiempo_total = time.time() - tiempo_inicio_total
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"especializado_{Utils.crear_nombre_archivo(modelo_info, umbral_config, timestamp)}"
    archivo_completo = config.OUTPUT_PATH / nombre_archivo

    metadatos = {
        'metadatos': {
            'version': 'librerias_especializadas_mahotas_skimage',
            'modelo': modelo_info,
            'configuracion_umbrales': {
                'nombre': umbral_config,
                'descripcion': umbral_info['descripcion'],
                'valores': umbral_info['valores']
            },
            'timestamp': timestamp,
            'tiempo_total_segundos': tiempo_total,
            'total_imagenes': len(imagenes),
            'imagenes_exitosas': len(resultados),
            'librerias_usadas': ['mahotas', 'scikit-image', 'opencv-python']
        },
        'resultados': resultados
    }

    Utils.guardar_json(metadatos, str(archivo_completo))

    # Mostrar resumen avanzado
    procesador.mostrar_resumen_avanzado(resultados, umbral_config)

    print(f"\n⏱️  TIEMPO TOTAL: {tiempo_total:.1f} segundos")
    print(f"📁 Archivo guardado como: {nombre_archivo}")
    print(f"🎯 Características extraídas: Haralick, LBP, GLCM, Zernike, Multi-scale")

    detector.limpiar()
    return str(archivo_completo)


def main_con_caracteristicas_avanzadas():

    print(f"\n{'='*100}")
    print(f"🎯 EVALUACIÓN COMPLETA MASK2FORMER + CARACTERÍSTICAS AVANZADAS")
    print(f"{'='*100}")
    print(f"📊 Modelos disponibles: {len(config.MODELOS_INFO)}")
    print(f"⚙️  Configuraciones de umbral: {len(config.UMBRALES)}")
    print(f"🔬 Características: Mahotas + Scikit-image + OpenCV")

    total_combinaciones = len(config.MODELOS_INFO) * len(config.UMBRALES)
    print(f"🔄 Total de combinaciones: {total_combinaciones}")

    combinacion_actual = 0

    try:
        for i, modelo_info in enumerate(config.MODELOS_INFO):
            for umbral_config in config.UMBRALES.keys():
                combinacion_actual += 1

                print(f"\n{'='*100}")
                print(f"🔄 COMBINACIÓN {combinacion_actual}/{total_combinaciones}")
                print(f"{'='*100}")

                resultado = ejecutar_con_librerias_especializadas(i, umbral_config)

                if resultado:
                    print(f"✅ Combinación {combinacion_actual} completada exitosamente")
                    print(f"📁 Archivo: {os.path.basename(resultado)}")
                else:
                    print(f"❌ Error en combinación {combinacion_actual}")

                if combinacion_actual < total_combinaciones:
                    print(f"⏳ Pausa de 3 segundos para liberar memoria...")
                    time.sleep(3)

    except KeyboardInterrupt:
        print(f"\n\n⚠️  EVALUACIÓN INTERRUMPIDA POR EL USUARIO")
        print(f"📊 Progreso: {combinacion_actual}/{total_combinaciones} combinaciones completadas")
    except Exception as e:
        print(f"\n❌ ERROR DURANTE LA EVALUACIÓN: {str(e)}")
        print(f"📊 Progreso: {combinacion_actual}/{total_combinaciones} combinaciones completadas")

    print(f"\n🎉 EVALUACIONES FINALIZADAS")
    print(f"📁 Todos los resultados guardados en: {config.OUTPUT_PATH}")
    print(f"🔬 Con características avanzadas extraídas usando librerías especializadas")

In [12]:
if __name__ == "__main__":
    main_con_caracteristicas_avanzadas()


🎯 EVALUACIÓN COMPLETA MASK2FORMER + CARACTERÍSTICAS AVANZADAS
📊 Modelos disponibles: 3
⚙️  Configuraciones de umbral: 4
🔬 Características: Mahotas + Scikit-image + OpenCV
🔄 Total de combinaciones: 12

🔄 COMBINACIÓN 1/12

🚀 EVALUACIÓN CON LIBRERÍAS ESPECIALIZADAS
📋 CONFIGURACIÓN:
   🤖 Modelo: swin-large-coco-instance
   🔧 Tipo: INSTANCIA
   📚 Librerías: Mahotas + Scikit-image + OpenCV
   ⚙️  Umbrales: Detecta cambios mínimos - Máxima sensibilidad
📁 Encontradas 7 imágenes en /content/drive/MyDrive/TFM/mask2former/imagenes

🤖 CARGANDO MODELO:
   📌 Nombre: swin-large-coco-instance
   🔧 Tipo: INSTANCIA
   🏗️  Arquitectura: Swin-Large
   📊 Dataset: COCO
   💡 Descripción: Segmentación por instancia - Mayor precisión
   🖥️  Dispositivo: cuda
   ⏳ Cargando procesador...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


   ⏳ Cargando modelo...


...

   ✅  0 personas | 98.7ms | Características ✓

📷 [  6/  7] persona_medioplano_2.jpg
   🔍 Extrayendo características avanzadas...


Procesando con librerías especializadas:  86%|████████▌ | 6/7 [01:45<00:16, 16.77s/it]

   ✅  0 personas | 98.5ms | Características ✓

📷 [  7/  7] persona_primerisimoplano.jpg
   🔍 Extrayendo características avanzadas...


Procesando con librerías especializadas: 100%|██████████| 7/7 [01:54<00:00, 16.39s/it]

   ✅  0 personas | 98.4ms | Características ✓
💾 Archivo guardado: especializado_mask2former_instancia_coco_swinsmall_baja_sensibilidad_20250910_201643.json

📊 RESUMEN DE PROCESAMIENTO:
   ✅ Imágenes procesadas exitosamente: 7/7
   🤖 Modelo usado: swin-small-coco-instance
   🔧 Tipo: INSTANCIA
   ⚙️  Configuración umbrales: Solo detecciones muy confiables

📈 RESULTADOS POR UMBRAL:
   🎯 Umbral  0.3000:   0 personas |   0/7 imágenes (  0.0%)
   🎯 Umbral  0.5000:   0 personas |   0/7 imágenes (  0.0%)
   🎯 Umbral  0.7000:   0 personas |   0/7 imágenes (  0.0%)

🧬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:
   📊 Haralick promedio: 1349.710 ± 538.745
   ⚡ GLCM Contrast: 41.770 ± 22.391
   📐 Esquinas promedio: 4016.9 ± 733.9
   🔬 Usando: Mahotas + Scikit-image + OpenCV

⏱️  TIEMPO TOTAL: 114.8 segundos
📁 Archivo guardado como: especializado_mask2former_instancia_coco_swinsmall_baja_sensibilidad_20250910_201643.json
🎯 Características extraídas: Haralick, LBP, GLCM, Zernike, Multi-scale
🧹 Liberando m